# Solving Quadratic Assignment Problem as a QUBO model with quantum annealing

The notebook contains the steps to read a QAP instance, transform it into a QUBO model and send it to a D-wave the quantum computer to solve it with quantum annealing technique.

## QAP instances

QAPLIB is a large library of QAP instances based of real-world problems, information on the research of the QAP and contributions towards spreading the scientific knowledge about the QAP to the world. It can be accessed <a href="http://anjos.mgi.polymtl.ca/qaplib/">here</a>.

The first step is to read the data from an instance of the QAP problem. In this case the data corresponds to the amount of locations and facilities, the flow chart between facilities and the distance between locations. The data is a file with the following format:

n

A

B

The 'n' corresponds to the amount of facilities/locations, while A and B are either the flow chart between facilities or the distance between locations. Due to the problems' characteristics, the order in which each one of the matrices is given is inconsequential.

In [1]:
import numpy as np
import os.path

# Reads a QAP instance from the QAPLIB and return the flow and distance matrices and the size n of the problem
def read_QAP_instance(raw_data):
    
    # Check if the file exists
    if not os.path.isfile(raw_data):
        print("File not found\n")
        return None
    
    # Every bit of information is taken from the file
    with open(raw_data) as f:
        int_array = list(map(int,f.read().strip().split()))
        #print(int_array)
    
    # The number of locations/facilities
    n = int_array[0]
    # The first matrix A obtained from the file
    A = int_array[1:(1+n**2)]
    A = np.reshape(A, (n,n))
    # The second matrix B obtained from the file
    B = int_array[(1+n**2):]
    B = np.reshape(B, (n,n))
    
    # The information extracted from the file is printed in an organized manner
    print()
    print("-------------------------------------------------------------------")
    print("Instance: ", raw_data)
    print()
    print("The number of locations/facilities of the instance is {}".format(n))
    print()
    print("The matrix A of the instance:")
    print(A)
    print()
    print("The matrix B of the instance:")
    print(B)
    print()
    
    return (n, A, B)

        

## Build the QUBO model

The QAP is now transformed into a QUBO model equivalent. In particular, the objective function of the QAP is condensed into a Q matrix. The Q matrix is a $n^2 \times n²$ matrix, where each position corresponds to the quadratic coefficient of any assignment of a facility to a location to any other of that sort. Moreover, the Q matrix also considers the constraints of the problem. This is the QAP model:

$$ \min \sum_{i=1}^{n}\sum_{j=1}^{n}\sum_{k=1}^{n}\sum_{l=1}^{n} f_{ij}d_{kl}x_{ik}x_{jl} \\
    \sum_{i=1}^{n} x_{ij} = 1 \ \ j \in S_n \\
    \sum_{j=1}^{n} x_{ij} = 1 \ \ i \in S_n \\
    x_{ij}\in \{0,1\} \ \ i,j \in S_n$$
    
The main purpose of this step is to transform that QAP model into this QUBO equivalent model:

$$ \min y=x^tQx $$

For that purpose, not only the objective function must be transformed but also the constraints. The objective function can be straightforwardly placed into the Q matrix by adding the coefficients of compatible decision variables in their respective position. Constraints, however, are included with penalty terms, so that the incompatible decision variables are penalized. For the later, a discrete parameter P (the Lagrange parameter) that is adjusted to capture the strength of each constraint is defined.

In this case, every constraint is necessary to be met since the same facility can't be placed in two different locations and a location can't hold two different facilities. Therefore, the parameter P will be adjusted as the largest coefficient of the objective function, so that considering that possibility is equivalent to having being placed in the worst possible place for the specific amount of flow between the facilities.



In [2]:
import dimod

# Transforms the QAP model into a QUBO model. The parameter n is used to indicate the subset of the problem taken
def generate_qubo_model(n, A, B, P=None):
    
    # The Q matrix is initialized
    Q = np.zeros(shape=(n*n,n*n))
    
    if P is None:
        P = max(map(max, A)) * max(map(max, B))/2
    offset = 2*n*P
    
    # The Q matrix is filled
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    # These correspond to the diagonal of the matrix
                    if i==j and k==l:
                        Q[i*n+k, j*n+l] = -2*P
                    # These correspond to the decision variables that can't occur at the same time, which is
                    # when a facility is in two locations or two facilities are in the same location
                    elif i==j or k==l:
                        Q[i*n+k, j*n+l] = P
                    # Valid pairs of decision variables come directly from the original objective function
                    else:
                        Q[i*n+k, j*n+l] = (A[i][j] * B[k][l]) / 2
    
    bqm = dimod.BinaryQuadraticModel.from_numpy_matrix(Q, offset=offset)
    (Q, offset) = bqm.to_qubo()
    return (Q, offset)
    

## Connecting to the QPU

With the coefficients of the model being gathered in the Q matrix, the next step involves quantum annealing in the quantum computers available from D-wave systems. 

In [3]:
# Library to interact with the QPU
from dwave.system.samplers import DWaveSampler
# Library to embed our problem onto the QPU physical graph
from dwave.system.composites import EmbeddingComposite

# Send the QUBO model to the D-Wave QPU to use quantum annealing and solve the QUBO model. The token must be specified.
def send_QUBO_model(Q, token, chainstrength=1, numruns=10000):

    response = EmbeddingComposite(DWaveSampler(token=token, solver={'qpu': True})).sample_qubo(Q, chain_strength=chainstrength, num_reads=numruns)
    print()
    print("QPU call complete using", response.info['timing']['qpu_access_time'], "microseconds of QPU time.")
    print()
    return response


## Unembedding the response

After receiving the response from the D-Wave server, the minor embedding of the QUBO model has to be reverted to obtain the values of the logical qubits from the physical qubits. This is achieved by the optimization process at the end of the execution.

The QPU returns solutions ordered from the lowest energy ones to the largest ones. The results are displayed along with their energy and the original objective function value by adding the offset. The chain break frequency is also displayed to see whether the chain strength was high enough. The solution can be checked to see wheter it's a feasible solution (constraints satisfied) or not. In case one of the facilities hasn't been assigned a location, the value in the solution is -1 for that facility.

In [4]:

def display_sample_solutions(response,n,offset):
    
    R = iter(response)
    E = iter(response.data())
    n_solution = 0
    solutions = {}
    energies = {}
    chain_break = {}
    objective_f = {}
    
    for i in response:
        n_solution += 1
        solution = next(R)
        sample = next(E)
        energies[n_solution] = (sample.energy)
        chain_break[n_solution] = (sample.chain_break_fraction)
        objective_f[n_solution] = sample.energy + offset
        solutions[n_solution] = [-1 for v in range(n)]
        for k in range(n):
            for l in range(n):
                if solution[n*k+l] == 1:
                    solutions[n_solution][k] = l
    
    print("Energies     ObjectiveF  CBFrequency  Solution")
    for i in range(10):
        print(energies[i+1],"   ", objective_f[i+1], "   ", chain_break[i+1],"   ", solutions[i+1])
        

## The whole process

The entire process has been implemented from reading an instance and transforming the QAP model into the QUBO model to sending the QUBO model and processing the response from the server to diplay the result. Several instances of QAP will be run, for which the best 10 solutions will be printed. The instances are stored in the instances folderand are directly extracted from QAPLIB.

The Lagrange parameter and the chain strength have been previously calculated to obtain feasible and good quality results.

In [5]:
from os import listdir

def solve_instances():
    
    prefix = "./instances/"
    instances = listdir(prefix)
    print("Instances: ", instances)
    parameters = [12000,12000]
    chain_strengths = [30000,30000]
    n=8
    
    for i in range(len(parameters)):
        (n, A, B) = read_QAP_instance(prefix+instances[i])
        # A subset of the problem is taken in case it can exceed the minor embedding to the chimera graph for the QPU.
        if n > 8:
            n = 8
        (Q, offset) = generate_qubo_model(n, A, B, P=parameters[i])
        response = send_QUBO_model(Q,'DEV-464cff42c60e9e429494e2ff96d97e0b5f8f4e91',chainstrength=30000)
        display_sample_solutions(response,n,offset)
        
        
        

solve_instances()
    

Instances:  ['chr12a.dat', 'chr15b.dat', 'chr12c.dat', 'chr15c.dat', 'chr12b.dat', 'chr15a.dat']

-------------------------------------------------------------------
Instance:  ./instances/chr12a.dat

The number of locations/facilities of the instance is 12

The matrix A of the instance:
[[ 0 90 10 23 43  0  0  0  0  0  0  0]
 [90  0  0  0  0 88  0  0  0  0  0  0]
 [10  0  0  0  0  0 26 16  0  0  0  0]
 [23  0  0  0  0  0  0  0  0  0  0  0]
 [43  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 88  0  0  0  0  0  0  1  0  0  0]
 [ 0  0 26  0  0  0  0  0  0  0  0  0]
 [ 0  0 16  0  0  0  0  0  0 96  0  0]
 [ 0  0  0  0  0  1  0  0  0  0 29  0]
 [ 0  0  0  0  0  0  0 96  0  0  0 37]
 [ 0  0  0  0  0  0  0  0 29  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 37  0  0]]

The matrix B of the instance:
[[ 0 36 54 26 59 72  9 34 79 17 46 95]
 [36  0 73 35 90 58 30 78 35 44 79 36]
 [54 73  0 21 10 97 58 66 69 61 54 63]
 [26 35 21  0 93 12 46 40 37 48 68 85]
 [59 90 10 93  0 64  5 29 76 16  5 76]
 [72 58 97 12 64